In [29]:
import os
import re
import json
import pandas as pd
from pprint import pprint

In [30]:
RAW_DATASET_PATH = "/root/sample_gen/data/raw/mmlu"
DATASET_PROMPT_TEMPLATES = "/root/sample_gen/data/raw/mmlu/mmlu-cot.json"
PROCESSED_DATASET_PATH = "/root/sample_gen/data/processed/mmlu/mmlu_cot.json"
SPLITS = ["dev","test","val"]
SPLIT_MAP = {"dev":"train","test":"test","val":"val"}

In [31]:
data = json.load(open(DATASET_PROMPT_TEMPLATES, 'r'))
len(data)

57

In [32]:
def remove_first_sentence(input_string):
    # Split the string into sentences using period as the delimiter
    sentences = input_string.split('.')

    # Remove the first sentence
    if len(sentences) > 1:
        del sentences[0]

    # Join the remaining sentences to form the updated string
    updated_string = '. '.join(sentences)

    return updated_string

few_shot_prompt_templates ={}
for key, val in list(data.items()):
    new_val = remove_first_sentence(val)
    temp_val =  new_val.replace('\n', '').replace('\n\n', '').replace('\n\n\n', '')
    mod_val = temp_val.replace('Q:', ' <END_ANSWER> \n <START_QUESTION> Question: ').replace('A:', ' <END_QUESTION> \n <START_ANSWER> Answer: ')
    clen_val = mod_val.strip()
    final_val = clen_val[14:]+" "+str("<END_ANSWER>")
    few_shot_prompt_templates[key.strip()] = final_val
    
assert len(data) == len(few_shot_prompt_templates)

In [33]:
print(few_shot_prompt_templates["global_facts"])

 <START_QUESTION> Question:  As of 2017, how many of the world’s 1-year-old children today have been vaccinated against some disease? *(A) 80% (B) 60% (C) 40% (D) 20% <END_QUESTION> 
 <START_ANSWER> Answer:  Let's think step by step.  We refer to Wikipedia articles on global facts for help.  According to data published by the World Health Organization, the nummber of 1-year-old children vaccinated in 2017 exceeds 80%.  The answer is (A).  <END_ANSWER> 
 <START_QUESTION> Question:  As of 2019, about what percentage of Americans agree that the state is run for the benefit of all the people?(A) 31% (B) 46% (C) 61% (D) 76% <END_QUESTION> 
 <START_ANSWER> Answer:  Let's think step by step.  We refer to Wikipedia articles on global facts for help.  In 2019, about 46% percentage of Americans agree that the state is run for the benefit of all the people.  The answer is (B).  <END_ANSWER> 
 <START_QUESTION> Question:  As of 2019, about what percentage of Russians say it is very important to hav

In [34]:
def add_question_to_prompt_template(q,template):
    out = q[0]+" "
    new_template = template
    for bullet, option in zip(["A","B","C","D"],[1,2,3,4]):
        out += f"({bullet}) {q[option]} "
    out += "\n"
    temp_q =out.replace('\n', '').replace('\n\n', '').replace('\n\n\n', '')
    new_template += f"\n<START_QUESTION> Question: {temp_q.strip()} <END_QUESTION> \n <START_ANSWER> Answer: "
    return new_template

def return_answer(q):
    return q[5]

In [35]:
all_data = []
for split in SPLITS:
    for fp in os.listdir(os.path.join(RAW_DATASET_PATH,split)):
        fpath = os.path.join(RAW_DATASET_PATH,split,fp)
        if not os.path.isfile(fpath):
            continue
        df = pd.read_csv(fpath, header=None).to_dict(orient="records")
        subject = '_'.join(fp.split('.')[0].strip().split('_')[:-1]).strip()
        for qidx, q in enumerate(df):
            all_data.append({
                "uid": f"{SPLIT_MAP[split]}-{subject}-{qidx}",
                "split": split,
                "subject": subject,
                "raw": q,
                "answer": return_answer(q),
                "few_shot_cot_prompt": add_question_to_prompt_template(q,few_shot_prompt_templates[subject]),
                
            })

In [36]:
json.dump(all_data,open(PROCESSED_DATASET_PATH,'w'),indent=3)

In [37]:
print(len(all_data))
pprint(all_data[1560])

15858
{'answer': 'C',
 'few_shot_cot_prompt': ' <START_QUESTION> Question:  This question refers to '
                        'the following information. I come not to urge '
                        'personal claims, nor to seek individual benefits; I '
                        'appear as the advocate of those who cannot plead '
                        'their own cause; I come as the friend of those who '
                        'are deserted, oppressed, and desolate.  In the '
                        'Providence of God, I am the voice of the maniac whose '
                        'piercing cries from the dreary dungeons of your jails '
                        'penetrate not your Halls of Legislation.  I am the '
                        'Hope of the poor crazed beings who pine in the cells, '
                        'and stalls, and cages, and waste rooms of your '
                        'poor-houses.  I am the Revelation of hundreds of '
                        'wailing, suffering cre